In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from database.market import Market
from datetime import datetime, timedelta
from tqdm import tqdm
from extractor.tiingo_extractor import TiingoExtractor

In [2]:
market = Market()
market.connect()
dates = market.retrieve_date_range()
sp5 = market.retrieve("sp500")
market.disconnect()

In [3]:
stuff = pd.to_datetime(dates["date"])

In [4]:
start = (stuff.max() + timedelta(days=1)).strftime("%Y-%m-%d")
end = datetime.now().strftime("%Y-%m-%d")
print(start,end)

2021-12-11 2021-12-14


In [5]:
market.connect()
for ticker in tqdm(sp5["Symbol"].unique()):
    try:
        if "." in ticker:
            ticker = ticker.replace(".","-")
        data = TiingoExtractor.extract(ticker,start,end)
        data["ticker"] = ticker
        market.store("prices",data)
    except Exception as e:
        print(str(e))
market.disconnect()

 16%|████████████████                                                                                    | 81/505 [00:25<01:00,  7.02it/s]

Expecting value: line 1 column 1 (char 0)
'NoneType' object does not support item assignment


 61%|████████████████████████████████████████████████████████████▏                                      | 307/505 [01:03<00:41,  4.81it/s]

project_market prices documents must be a non-empty list


100%|██████████████████████████████████████████████████████████████████████████████████████████████████▌| 503/505 [01:38<00:00,  6.91it/s]

Expecting value: line 1 column 1 (char 0)
'NoneType' object does not support item assignment
Expecting value: line 1 column 1 (char 0)
'NoneType' object does not support item assignment


100%|███████████████████████████████████████████████████████████████████████████████████████████████████| 505/505 [01:39<00:00,  5.09it/s]

Expecting value: line 1 column 1 (char 0)
'NoneType' object does not support item assignment
Expecting value: line 1 column 1 (char 0)
'NoneType' object does not support item assignment


In [6]:
data

In [7]:
analysis = []
market.connect()
for ticker in tqdm(sp5["Symbol"].unique()):
    report = {}
    try:
        test = market.retrieve_ticker_prices("prices",ticker)
        test["date"] = pd.to_datetime(test["date"])
        test["year"] = [x.year for x in test["date"]]
        test = test[test["year"]==test["year"].max()].sort_values("date")      
        test["quarter"] = [x.quarter for x in test["date"]]
        test["week"] = [x.week for x in test["date"]]
        test["month"] = [x.month for x in test["date"]]
        periods = ["year","quarter","month","week"]
        ep = test.tail(1)["adjClose"].item()
        # periods = ["year"]
        for period in periods:
            current = test.tail(1)[period].item()
            window = test[(test[period]==current)].sort_values("date")
            sp = window.iloc[0]["adjClose"].item()
            gain = (ep - sp) / sp
            report[f"{period}_gain"] = gain
        report["day_gain"] = (ep - test.tail(2).iloc[0]["adjClose"].item())/test.tail(2).iloc[0]["adjClose"].item()
        report["ticker"] = ticker
        analysis.append(report)
    except Exception as e:
        print(str(e))
market.disconnect()

 13%|█████████████                                                                                       | 66/505 [00:04<00:32, 13.68it/s]

'date'


 16%|███████████████▊                                                                                    | 80/505 [00:05<00:27, 15.29it/s]

'date'


100%|███████████████████████████████████████████████████████████████████████████████████████████████████| 505/505 [00:36<00:00, 13.86it/s]

'date'
'date'
'date'
'date'


In [8]:
a = pd.DataFrame(analysis).merge(sp5.rename(columns={"Symbol":"ticker"})[["ticker","Security","GICS Sector"]],on="ticker",how="left")
a.to_csv("analysis.csv")

In [9]:
period = "day"
a.sort_values(f"{period}_gain",ascending=False).head(50)

,year_gain,quarter_gain,month_gain,week_gain,day_gain,ticker,Security,GICS Sector
315,1.434351,-0.202586,-0.124336,0.000000,0.058080,MRNA,Moderna,Health Care
74,-0.015268,-0.007730,0.074613,0.000000,0.047729,BMY,Bristol-Myers Squibb,Health Care
368,0.544941,0.285814,0.009510,0.000000,0.045851,PFE,Pfizer,Health Care
296,-0.283037,-0.038174,0.169144,0.000000,0.041546,MKTX,MarketAxess,Financials
243,-0.215616,-0.005864,0.043873,0.000000,0.036850,INCY,Incyte,Health Care
63,0.018579,0.033227,0.046096,0.000000,0.032004,BDX,Becton Dickinson,Health Care
95,-0.004518,0.090560,0.095701,0.000000,0.031464,CERN,Cerner,Health Care
110,-0.339114,-0.225030,0.067454,0.000000,0.030916,CTXS,Citrix Systems,Information Technology
62,0.048858,0.023496,0.108151,0.000000,0.028939,BAX,Baxter International,Health Care
343,0.128638,-0.016970,0.122753,0.000000,0.028564,NRG,NRG Energy,Utilities


In [10]:
a.groupby("GICS Sector").mean().sort_values("day_gain",ascending=False).head(20)

,year_gain,quarter_gain,month_gain,week_gain,day_gain
GICS Sector,,,,,
Consumer Staples,0.120194,0.065233,0.076189,0.000000,0.012013
Utilities,0.154381,0.077800,0.065640,0.000000,0.011062
Real Estate,0.482476,0.089221,0.066180,0.000000,0.009282
Health Care,0.190361,0.010734,0.050644,0.000000,0.005823
Materials,0.272653,0.064769,0.046666,0.000000,-0.008928
Industrials,0.266080,0.052735,0.048987,0.000000,-0.009372
Information Technology,0.251614,0.060833,0.034307,0.000000,-0.009381
Communication Services,0.087939,-0.096260,0.033180,0.000000,-0.009405
Financials,0.345442,0.008368,0.032712,0.000000,-0.013783
